In [8]:
import pandas as pd
from glob import glob
from PyPDF2 import PdfReader
import time
from IPython.display import Markdown, display


# Baseline Data Extraction

In [2]:
def extract_pdf_text(path):
    data = {'pdf_path': path}
    text = ""
    reader = PdfReader(path)
    for page in reader.pages:
        text += page.extract_text() + "\n"
    data['text'] = text
    return pd.DataFrame.from_dict(data, orient='index').T

In [3]:
all_plans = []
for pdf in glob('./diet_samples/**/*.pdf'):
    all_plans.append(extract_pdf_text(pdf))

master_df = pd.concat(all_plans, ignore_index=True)

In [4]:
from langchain_core.messages import SystemMessage, HumanMessage

def improve_baseline_text(raw_text, model):
    system_message = SystemMessage(
        content="""
You are given the text extracted from a Strongr Fastr PDF meal plan. 
Your task is to carefully structure all the information into clear Markdown sections.

Follow this exact format:

# Meal Plan Overview
- Calories: [value]
- Protein: [value]
- Carbohydrates: [value]
- Fat: [value]
- Fiber: [value]
- Other notes if available

# Daily Meal Tables
For each day (Day 1 through Day 7), create a Markdown table with these columns:
| Meal | Recipe Name | Calories | Protein (g) | Carbs (g) | Fat (g) | Fiber (g) |

# Grocery List
- Organize the grocery list into categories (e.g., Proteins, Carbs, Vegetables, Fruits, Fats, Misc).
- Use Markdown bullet points.

# Recipes
For each recipe:
## [Recipe Name]
- Ingredients: list with quantities
- Instructions: step-by-step
- Nutrition facts (Calories, Protein, Carbs, Fat, Fiber)

# Notes
- Capture any additional information included in the PDF.

Important:
- Keep all formatting in clean Markdown.
- Preserve tables where possible (don’t convert to plain text).
- Do not summarize or skip items. Extract everything.

"""
    )

    user_message = HumanMessage(
        content=(
            "Here is a raw baseline workout plan. Please reorganize it into a well-structured, human-readable weekly plan following the system instructions:\n"
            f"{raw_text}"
        )
    )

    response = model.invoke([system_message, user_message])
    
    return response.content



In [9]:
from dotenv import load_dotenv
import os

load_dotenv()

avalai_api_key = os.getenv("AVALAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [24]:
from langchain_openai import ChatOpenAI

AVALAI_BASE_URL = "https://api.avalai.ir/v1/"
MODEL = "gpt-4.1-mini"
MODEL_JUDGE ="gpt-4.1-mini"

llm = ChatOpenAI(
    model=MODEL,
    base_url=AVALAI_BASE_URL,
    api_key=avalai_api_key,
    temperature=1,
)
llm_judge = ChatOpenAI(
    model=MODEL_JUDGE,
    base_url=AVALAI_BASE_URL,
    api_key=avalai_api_key,
    temperature=0,
)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)

MODEL_NAME = "gemini-2.5-flash-preview-05-20"

gemini_chat = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0, base_url=gemini_api_key, api_key=avalai_api_key, rate_limiter=rate_limiter)
# gemini_chat_half = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=0.5, api_key=gemini_api_key, rate_limiter=rate_limiter)
# gemini_chat_1 = ChatGoogleGenerativeAI(model=MODEL_NAME, temperature=1, api_key=gemini_api_key, rate_limiter=rate_limiter)


/tmp/ipykernel_23070/1951523411.py:4: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(requests_per_second=0.1, check_every_n_seconds=10, max_bucket_size=10)


In [8]:
baseline_plans = []
for m in master_df['text']:
    baseline_plans.append(improve_baseline_text(m, llm))
    # time.sleep(10)

In [13]:
master_df['improved_text'] = baseline_plans
master_df.to_csv('./data/diets_data.csv', index=False)

In [7]:
display(Markdown(master_df.loc[0, 'improved_text']))

# Meal Plan Overview
- Calories: 1000-1016 kcal per day
- Protein: Approximately 89-112g (varies slightly by day)
- Carbohydrates: Approximately 29-85g (varies by day)
- Fat: Approximately 3-44g (varies by day)
- Fiber: 10-19g per day
- Other notes: This is a macro-focused meal plan with a variety of lean proteins, vegetables, and controlled carbohydrate sources.

# Daily Meal Tables

| Day | Meal       | Recipe Name                        | Calories | Protein (g) | Carbs (g) | Fat (g) | Fiber (g) |
|-------|------------|-------------------------------------|------------|--------------|-----------|---------|-----------|
| Day 1 | Breakfast  | Scrambled eggs with kale, tomatoes, rosemary | 203        | 15           | 7         | 12      | 2         |
|       | Lunch      | Lentils, Chicken sausage            | 430        | 40           | 29        | 15      | 11        |
|       | Dinner     | Flavored rice mix, Lemon garlic cod, Corn | 385        | 38           | 49        | 3       | 11        |
| Day 2 | Breakfast  | Scrambled eggs with kale, tomatoes, rosemary | 203        | 15           | 7         | 12      | 2         |
|       | Lunch      | Chicken breast, Lentils, Buttered green beans | 430        | 39           | 29        | 14      | 11        |
|       | Dinner     | Mozzarella & tomato salad, Brown rice, Teriyaki chicken | 370        | 36           | 21        | 15      | 11        |
| Day 3 | Breakfast  | Scrambled eggs with kale, tomatoes, rosemary | 203        | 15           | 7         | 12      | 2         |
|       | Lunch      | Chicken breast, Lentils, Buttered green beans | 430        | 39           | 29        | 14      | 11        |
|       | Dinner     | Mozzarella & tomato salad, Brown rice, Teriyaki chicken | 370        | 36           | 21        | 15      | 11        |
| Day 4 | Breakfast  | Sunflower seeds, Lowfat Greek yogurt | 245        | 17           | 17        | 11      | 10        |
|       | Lunch      | Chicken & spinach salad, Lentils, Marinaded chicken | 380        | 50           | 5         | 17      | 2         |
|       | Dinner     | Baked chicken with tomatoes & olives, Edamame & beet salad | 385        | 44           | 10        | 16      | 10        |
| Day 5 | Breakfast  | Sunflower seeds, Lowfat Greek yogurt | 245        | 17           | 17        | 11      | 10        |
|       | Lunch      | Chicken beet & carrot salad, Brown rice, Raspberries | 355        | 29           | 24        | 11      | 8         |
|       | Dinner     | Baked chicken with tomatoes & olives, Edamame & beet salad | 385        | 44           | 10        | 16      | 10        |
| Day 6 | Breakfast  | Creamy scrambled eggs, Raspberries | 182        | 13           | 2         | 14      | 0         |
|       | Lunch      | Lentils, Marinated chicken, Mixed greens salad | 360        | 35           | 24        | 12      | 5         |
|       | Dinner     | Caprese salad, Lemon pepper chicken breast | 405        | 57           | 5         | 16      | 5         |
| Day 7 | Breakfast  | Creamy scrambled eggs, Raspberries | 182        | 13           | 2         | 14      | 0         |
|       | Lunch      | Lentils, Marinated chicken, Mixed greens salad | 360        | 35           | 24        | 12      | 5         |
|       | Dinner     | Caprese salad, Lemon pepper chicken breast | 405        | 57           | 5         | 16      | 5         |

# Grocery List

## Spices and Herbs
- Salt: 1/4 oz (8g)
- Rosemary, dried: 1/4 tbsp (1g)
- Balsamic vinegar: 1 tbsp (15mL)
- Fresh basil: 1/2 oz (15g)
- Black pepper: 1.5 g (2g)
- Chili powder: 2 tsp (5g)
- Thyme, dried: dash
- Lemon pepper: 1 tbsp (7g)
- Italian seasoning: dash
- Garlic: 1.5 cloves (5g)

## Beverages
- Water: 1/3 gallon (1306mL)

## Legumes and Legume Products
- Lentils, raw: 1 cup (208g)

## Sausages and Luncheon Meats
- Chicken sausage, cooked: 2 links (168g)

## Meals, Entrees, and Side Dishes
- Flavored rice mix: 1/4 pouch (~5.6 oz) (40g)

## Other
- Italian seasoning: 1.5 dash (1g)
- Teriyaki sauce: 4 tbsp (60mL)
- Mixed greens: 1.5 packages (243g)
- Olive oil: 1/2 oz (18mL)
- Salad dressing: 6.5 tbsp (96mL)
- Marinade sauce: 4 tbsp (60mL)

## Vegetables and Vegetable Products
- Garlic: 1.5 cloves (5g)
- Kale leaves: 3 cups, chopped (120g)
- Tomatoes: 5.5 medium whole (685g)
- Frozen green beans: 1.33 cups (161g)
- Beets, precooked: 1/3 lbs (157g)
- Edamame, frozen, shelled: 1/2 cup (59g)
- Fresh spinach: 2.5 cups (75g)
- Carrots: 1/4 medium (15g)
- Green olives: 12 large (53g)
- Cherry tomatoes: 4-12 (depending on recipe)

## Fruits and Fruit Juices
- Lemons: 3/8 large (32g)
- Raspberries: 2 cups (246g)

## Finfish and Shellfish Products
- Cod, raw: 6 oz (170g)

## Dairy and Egg Products
- Eggs: 10 large (500g)
- Fresh mozzarella cheese: 1/4 lbs (99g)
- Lowfat flavored Greek yogurt: 2 containers (300g)
- Whole milk: 2 tbsp (31mL)

## Fats and Oils
- Oil (vegetable or olive): 1 oz (36mL)
- Balsamic vinaigrette: 1.5 oz (45mL)
- Olive oil: 1/2 oz (18mL)
- Salad dressing: 6.5 tbsp (96mL)
- Marinade sauce: 4 tbsp (60mL)

## Cereal Grains and Pasta
- Brown rice: 4 tbsp (48g)

## Poultry Products
- Boneless skinless chicken breast, raw: 4 lbs (1785g)

## Nut and Seed Products
- Sunflower kernels: 1 oz (28g)
- Frozen corn kernels: 2/3 cup (91g)

# Recipes

## Scrambled eggs with kale, tomatoes, rosemary
- Ingredients:
  - Kale leaves: 1 cup (40g)
  - Eggs: 2 large (100g)
  - Tomatoes: 1/2 cup, chopped (90g)
  - Rosemary, dried: 2 dash
  - Balsamic vinegar: 1 tsp
  - Water: 3 tbsp
  - Oil: 1/2 tsp
- Instructions:
  1. Add oil to skillet over medium-high heat.
  2. Add kale, tomatoes, rosemary, water; cover and cook 4 min.
  3. Add eggs, scramble until cooked.
  4. Plate and optionally top with balsamic.
- Nutrition: 203 calories, 15g protein, 7g net carbs, 12g fat, 2g fiber

## Sunflower seeds
- Ingredients:
  - Sunflower kernels: 1 oz (28g)
- Instructions:
  - Serve as snack or topping.
- Nutrition: 245 calories, 17g protein, 17g fat, 1g carbs, 1g fiber

## Creamy scrambled eggs
- Ingredients:
  - Eggs: 2 large (100g)
  - Salt: dash
  - Black pepper: dash
  - Butter: 1 tsp
  - Whole milk: 1 tbsp
- Instructions:
  1. Beat eggs, milk, salt, pepper.
  2. Cook in butter over medium heat, scramble until thickened.
- Nutrition: 182 calories, 13g protein, 14g fat, 2g carbs, 0g fiber

## Lowfat Greek yogurt
- Ingredients:
  - 1 container (150g)
- Instructions:
  - Serve as is.
- Nutrition: 155 calories, 12g protein, 4g fat, 16g carbs, 2g fiber

## Chicken & spinach salad
- Ingredients:
  - Chicken breast, cooked: 1/2 lbs (213g)
  - Spinach: 2.5 cups (75g)
  - Salad dressing: 2 tbsp
  - Oil: 1.25 tsp
- Instructions:
  1. Cook chicken, slice.
  2. Toss with spinach and dressing.
- Nutrition: 380 calories, 50g protein, 17g fat, 5g carbs, 2g fiber

## Baked chicken with tomatoes & olives
- Ingredients:
  - Chicken: 6 oz (170g)
  - Cherry tomatoes: 6 (102g)
  - Olive oil: 1 tsp
  - Salt, chili powder, black pepper, basil, olives
- Instructions:
  1. Preheat oven to 425°F.
  2. Season chicken, top with tomatoes, olives, basil.
  3. Bake 25 min, check doneness.
- Nutrition: 300 calories, 40g protein, 12g fat, 4g carbs, 3g fiber

## Caprese salad
- Ingredients:
  - Mixed greens: 1/4 package (39g)
  - Cherry tomatoes: 1/2 cup (75g)
  - Fresh mozzarella: 1.5 oz (43g)
  - Basil: 4 tbsp leaves
  - Balsamic vinaigrette: 1 tbsp
- Instructions:
  1. Mix greens, basil, tomatoes.
  2. Top with mozzarella, drizzle dressing.
- Nutrition: 107 calories, 6g protein, 7g fat, 4g carbs, 1g fiber

## Lemon pepper chicken breast
- Ingredients:
  - Chicken: 8 oz (224g)
  - Lemon pepper: 1 tbsp
  - Olive oil: 1/2 tbsp
- Instructions:
  1. Rub chicken with oil and lemon pepper.
  2. Cook via stovetop, bake, or broil as preferred.
- Nutrition: 296 calories, 51g protein, 9g fat, 1g carbs, 1g fiber

## Edamame & beet salad
- Ingredients:
  - Edamame: 4 tbsp (30g)
  - Beets: 2 (50g)
  - Mixed greens: 1/2 cup (15g)
  - Balsamic vinaigrette: 1/2 tbsp
- Instructions:
  1. Cook edamame.
  2. Mix greens, edamame, beets, drizzle with dressing.
- Nutrition: 86 calories, 5g protein, 4g fat, 6g carbs, 3g fiber

---

# Notes
- The plan emphasizes lean proteins, vegetables, and controlled carbohydrate portions.
- Recipes are simple, with instructions provided for preparation.
- The plan is designed for a weekly cycle, with slight variations to maintain macro balance.
- Next week's plan can be generated on [Strongr Fastr](https://www.strongrfastr.com).

In [11]:
master_df = pd.read_csv('./data/evaluation/diet/diets_data.csv')
master_df

,pdf_path,text,improved_text
0,./diet_samples/calorie/1000_calorie_macro_meal...,Meal Plan \n- 1000 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 1000-1016 kc...
1,./diet_samples/calorie/1200_calorie_macro_meal...,Meal Plan \n- 1200 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 1200 kcal (a...
2,./diet_samples/calorie/1400_calorie_macro_meal...,Meal Plan \n- 1400 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 1400 kcal (a...
3,./diet_samples/calorie/1600_calorie_macro_meal...,Meal Plan \n- 1600 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 1600 kcal (a...
4,./diet_samples/calorie/1800_calorie_macro_meal...,Meal Plan \n- 1800 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 1800 kcal (a...
5,./diet_samples/calorie/2000_calorie_macro_meal...,Meal Plan \n- 2000 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 2000 kcal\n-...
6,./diet_samples/calorie/2200_calorie_macro_meal...,Meal Plan \n- 2200 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 2200 kcal (a...
7,./diet_samples/calorie/2400_calorie_macro_meal...,Meal Plan \n- 2400 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 2400 kcal (a...
8,./diet_samples/calorie/2600_calorie_macro_meal...,Meal Plan \n- 2600 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 2600 kcal (a...
9,./diet_samples/calorie/2800_calorie_macro_meal...,Meal Plan \n- 2800 calorie macro meal plan\nGr...,# Meal Plan Overview\n- Calories: 2800 kcal (a...


# Create Test Personas & Queries

In [19]:
from pydantic import BaseModel, Field
from typing import Literal

class UserRegistrationQuery(BaseModel):
    email: str = Field(description="User's email address")
    password: str = Field(description="A secure password")
    first_name: str = Field(description="User's given name")
    last_name: str = Field(description="User's family name")
    date_of_birth: str = Field(description="YYYY-MM-DD format")
    sex: Literal["M", "F", "O"] = Field(description="User's gender identity")
    height_cm: int = Field(description="Height in centimeters")
    weight_kg: int = Field(description="Weight in kilograms")
    activity_level: Literal["Sedentary", "Light", "Moderate", "Active", "Very Active"] = Field(description="Typical daily activity level")
    dietary_pref: str = Field(description="User's primary dietary preferences")
    query: str = Field(description="A 2-3 sentence natural-language request describing their goals")

structured_model = llm.with_structured_output(UserRegistrationQuery)


In [20]:
from langchain_core.messages import SystemMessage, HumanMessage

def create_prompt(plan, model):
    system_message = SystemMessage(
        content= """
You are an expert persona and query generator for a meal planning application. 
You will be given the full structured text of a meal plan (Markdown format with overview, daily tables, grocery list, recipes). 
Your job is to imagine a realistic user persona that would follow this plan and produce a UserRegistrationQuery object. 

Guidelines for persona creation:
- Email: Generate a realistic email using the user's name (e.g., "firstname.lastname@example.com").
- Password: Create a secure but generic password (e.g., "FitLife2025!").
- First and last name: Create natural names that match the persona’s context (e.g., cultural background inferred from meal preferences if possible).
- Date of birth: Ensure age is consistent with the plan (e.g., younger for bodybuilding/high calorie, older for low-carb/health-conscious). Format YYYY-MM-DD.
- Sex: Infer from context (if not obvious, pick randomly).
- Height and weight: Make values consistent with caloric needs (e.g., 180 cm / 80 kg for 2500 kcal plan; 160 cm / 55 kg for 1500 kcal plan).
- Activity level: Choose based on meal plan style (e.g., "Active" for 3100 kcal intermittent fasting, "Sedentary" for 1200 kcal low-carb).
- Dietary preference: Reflects the plan (e.g., "Vegetarian", "Paleo", "Low-carb", "Balanced macro").
- Query: A natural language query that this persona might ask when registering, based on the plan (e.g., "I want a high-protein vegetarian plan under 1500 calories", "I need a paleo intermittent fasting plan around 2400 calories", "I’m looking for a weight loss plan under 100g carbs per day").

Your response must be **valid JSON** that can be parsed into the following schema:

email: str = Field(description="User's email address")
password: str = Field(description="A secure password")
first_name: str = Field(description="User's given name")
last_name: str = Field(description="User's family name")
date_of_birth: str = Field(description="YYYY-MM-DD format")
sex: Literal["M", "F", "O"] = Field(description="User's gender identity")
height_cm: int = Field(description="Height in centimeters")
weight_kg: int = Field(description="Weight in kilograms")
activity_level: Literal["Sedentary", "Light", "Moderate", "Active", "Very Active"] = Field(description="Typical daily activity level")
dietary_pref: str = Field(description="User's primary dietary preferences")
query: str = Field(description="A 2-3 sentence natural-language request describing their goals")

Important:
- Use the meal plan content to ground your persona.
- Always output only the JSON object, no extra explanation.
"""
    )

    user_message = HumanMessage(
        content=(
            "Here is the meal plan:\n"
            f"{plan}\n\n"
            "Now respond as instructed above."
        )
    )

    response = model.invoke([system_message, user_message])
    return response


In [21]:
records = []
for m in master_df['improved_text']:
    records.append(create_prompt(m, structured_model).model_dump())
    # time.sleep(10)

records_df = pd.DataFrame(records)

In [27]:
records_df.to_csv('./data/diet_records.csv', index=False)

In [25]:
records_df = pd.read_csv('./data/evaluation/diet/diet_records.csv')
records_df

,email,password,first_name,last_name,date_of_birth,sex,height_cm,weight_kg,activity_level,dietary_pref,query,agent_plan,agent_context
0,jane.smith1@example.com,FitLife2025!,Jane,Smith,1990-05-15,F,165,60,Light,"Macro-focused, lean proteins, vegetables, cont...","I want a low-calorie, macro-focused meal plan ...",# Meal Plan Overview\n- Calories (average per ...,['Protein Foods\n\nHealthy Intake: Healthy eat...
1,jane.smith2@example.com,FitLife2025!,Jane,Smith,1990-05-15,F,165,60,Moderate,Balanced macro,"I'm looking for a low-carb, high-protein meal ...",# Meal Plan Overview\n- Calories (average per ...,['Importance of Calorie Balance Within Healthy...
2,jane.doe1@example.com,FitLife2025!,Jane,Doe,1990-05-15,F,165,60,Moderate,"High protein, moderate carbs, healthy fats","I want a high-protein, balanced meal plan arou...",# Meal Plan Overview\n- Calories (average per ...,['Importance of Calorie Balance Within Healthy...
3,jane.smith3@example.com,FitLife2025!,Jane,Smith,1990-05-15,F,165,60,Moderate,Balanced macro,"I want a low-calorie, balanced meal plan with ...",# Meal Plan Overview\n- Calories (average per ...,['Importance of Calorie Balance Within Healthy...
4,jane.smith4@example.com,FitLife2025!,Jane,Smith,1990-05-15,F,165,60,Moderate,Balanced macro,I want a balanced meal plan with lean proteins...,# Meal Plan Overview\n- Calories (average per ...,['About This Chapter\n\nThis chapter focuses o...
5,jane.doe2@example.com,FitLife2025!,Jane,Doe,1990-05-15,F,165,60,Moderate,Balanced macro,"I want a high-protein, balanced macro meal pla...",# Meal Plan Overview\n- Calories (average per ...,['About This Chapter\n\nThis chapter focuses o...
6,james.martin1@example.com,FitLife2025!,James,Martin,1990-06-15,M,180,80,Active,"Low-carb, high-protein, lean meats and vegetables","I want a low-carb, high-protein meal plan focu...",# Meal Plan Overview\n- Calories (average per ...,"['Food Groupa\n\nAmount[b] in the 2,000-Calori..."
7,jane.smith5@example.com,FitLife2025!,Jane,Smith,1990-05-15,F,165,60,Active,Balanced macro,"I want a high-protein, balanced macro meal pla...",# Meal Plan Overview\n- Calories (average per ...,['Protein Foods\n\nHealthy Intake: Healthy eat...
8,john.doe3@example.com,FitLife2025!,John,Doe,1990-05-15,M,180,80,Active,"High-protein, balanced macro",I want a high-protein meal plan with controlle...,# Meal Plan Overview\n- Calories (average per ...,['Protein Foods\n\nHealthy Intake: Healthy eat...
9,james.martin2@example.com,FitLife2025!,James,Martin,1990-05-15,M,180,80,Active,Balanced macro,I want a balanced meal plan with high protein ...,# Meal Plan Overview\n- Calories (average per ...,['Protein Foods\n\nHealthy Intake: Healthy eat...


# LLM as a Judge

In [14]:
from pydantic import BaseModel, Field
from typing import Literal

class PlanComparison(BaseModel):
    relevance: Literal["A", "B", "Tie"] = Field(
        description="Which plan better aligns with the persona's stated dietary goals and preferences"
    )
    completeness: Literal["A", "B", "Tie"] = Field(
        description="Which plan provides more thorough nutritional information, recipes, and meal coverage"
    )
    creativity: Literal["A", "B", "Tie"] = Field(
        description="Which plan shows more variety, novelty, or interesting meal ideas"
    )
    safety_and_evidence: Literal["A", "B", "Tie"] = Field(
        description="Which plan is safer, nutritionally balanced, and better supported by dietary guidelines"
    )
    clarity: Literal["A", "B", "Tie"] = Field(
        description="Which plan is more clearly written, structured, and easy to follow"
    )
    overall: Literal["A", "B", "Tie"] = Field(
        description="Overall better meal plan considering all criteria"
    )
    rationale: str = Field(
        description="A brief, objective explanation of the judgment"
    )


In [15]:
from langchain_core.messages import SystemMessage, HumanMessage

structured_model = llm_judge.with_structured_output(PlanComparison)

def judge(query, baseline_plan, agent_plan):
    system_message = SystemMessage(
        content="You are an expert dietitian and nutrition coach."
    )
    message = f"""Given the User Persona:
{query}

Plan A:
{baseline_plan}

Plan B:
{agent_plan}

Evaluate each meal plan on five dimensions: 
1) Relevance (alignment with dietary goals/preferences), 
2) Completeness (nutritional info, recipes, and meal coverage), 
3) Creativity (novelty and variety of meals), 
4) Safety and Evidence (nutritional balance, grounding in dietary science), 
5) Clarity (organization and readability). 

Do not favor based on presentation order, response length, or plan labels.

For each dimension, choose “A” if Plan A is stronger, “B” if Plan B is stronger, or “Tie” if they are equal. Then decide the overall winner in the same way. Finally, provide a concise rationale.

Output must conform to the PlanComparison schema."""
    
    user_message = HumanMessage(content=message)
    response = structured_model.invoke([system_message, user_message])
    return response


### Create agent plans

In [29]:
from db.db_manager import FitnessDB

db_manager = FitnessDB(db_path='./diet_temp1.db', use_dummy_data=False)

SQLite database './diet_temp1.db' created/verified with updated schema.


In [30]:
from werkzeug.security import generate_password_hash

for i, user in records_df.iterrows():
    db_manager.create_user(
        email = user['email'],
        password_hash = generate_password_hash(user['password']),
        first_name = user['first_name'],
        last_name = user['last_name'],
        date_of_birth = user['date_of_birth'],
        sex =  user['sex'],
        height_cm = user['height_cm'],
        weight_kg = user['weight_kg'],
        activity_level = user['activity_level'],
        dietary_pref = user['dietary_pref'],
    )

In [19]:
from agents.planner import PlannerGraph
from langchain_core.messages import HumanMessage
from db.retrievers import DocumentRetriever
from agents.enums import PlannerType

agent = PlannerGraph(
    llm = llm,
    db_manager = db_manager,
    book_retriever = DocumentRetriever('data/'),
    planner_type = PlannerType.DIET,
    num_results = 5,
    use_rag_data = True,
    summarize_logs = False # disable log summarization.
).compile()

Making the book retriever ready =====
table is already created=====
Book Retriever is ready ====


In [20]:
plans = []
contexts = []
for i, user in records_df.iterrows():
    response = agent.invoke({"messages": HumanMessage(content=user['query']), 'user_id':i+1}, config={"configurable": {"thread_id": i+1}})
    agent_plan = response['messages'][-1].content
    agent_context = response['contexts']
    # time.sleep(50)
    plans.append(agent_plan)
    contexts.append(agent_context)
records_df['agent_plan'] = plans
records_df['agent_context'] = contexts

Function 'node_log_summary'
Function 'node_summarize'
Returned: {'messages': [], 'user_id': 1} 
Returned: {'log_summary': None} 
Function 'node_search_doc'


Returned: {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JST3LzJZOOpdxg1dJlKOZCxU', 'function': {'arguments': '{"query": "low-calorie meal plan with lean proteins and vegetables"}', 'name': 'search_books'}, 'type': 'function'}, {'id': 'call_Caj4kZkfy3xAE5FUAwQ8M72p', 'function': {'arguments': '{"query": "macro-focused meal plan around 1000 calories"}', 'name': 'search_books'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 523, 'total_tokens': 585, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'text_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': None, 'image_tokens': None}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_368a354b49', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0aabdc1b-d851-4f28-883b-dd93b2955

Task exception was never retrieved
future: <Task finished name='Task-5' coro=<AsyncClient.aclose() done, defined at /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/httpx/_client.py:1978> exception=AssertionError()>
Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/httpx/_client.py", line 1985, in aclose
    await self._transport.aclose()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/httpx/_transports/default.py", line 406, in aclose
    await self._pool.aclose()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/httpcore/_async/connection_pool.py", line 353, in aclose
    await self._close_connections(closing_connections)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/httpcore/_async/connection_pool.py", line 343, in _close_connections
    with AsyncShieldCancellation():
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.

Returned: {'contexts': ['Source of Goal[a]\n\nChild 1-3\n\nFemale 4-8\n\nMale 4-8\n\nFemale 9-13\n\nMale 9-13\n\nFemale 14-18\n\nMale 14-18\n\nFemale 19-30\n\nCalorie Level(s) Assessed\n\n1,000\n\n1,200\n\n1,400, 1,600\n\n1,600\n\n1,800\n\n1,800\n\n2,200, 2,800, 3,200\n\n2,000\n\nMacronutrients\n\nProtein, g\n\nRDA\n\n13\n\n19\n\n19\n\n34\n\n34\n\n46\n\n52\n\n46\n\nProtein, % kcal\n\nAMDR\n\n5-20\n\n10-30\n\n10-30\n\n10-30\n\n10-30\n\n10-30\n\n10-30\n\n10-35\n\nCarbohydrate, g\n\nRDA\n\n130\n\n130\n\n130\n\n130\n\n130\n\n130\n\n130\n\n130\n\nCarbohydrate, % kcal\n\nAMDR\n\n45-65\n\n45-65\n\n45-65\n\n45-65\n\n45-65\n\n45-65\n\n45-65\n\n45-65\n\nDietary Fiber, g\n\n14 g/ 1,000 kcal\n\n14\n\n16.8\n\n19.6\n\n22.4\n\n25.2\n\n25.2\n\n30.8\n\n28\n\nAdded Sugars, % kcal\n\nDGA\n\n<10%\n\n<10%\n\n<10%\n\n<10%\n\n<10%\n\n<10%\n\n<10%\n\n<10%\n\nTotal Fat, % kcal\n\nAMDR\n\n30-40\n\n25-35\n\n25-35\n\n25-35\n\n25-35\n\n25-35\n\n25-35\n\n20-35\n\nSaturated Fat, % kcal\n\nDGA\n\n<10%\n\n<10%\n\n<10%

In [31]:
from agents.planner import PlannerGraph
from langchain_core.messages import HumanMessage
from db.retrievers import DocumentRetriever
from agents.enums import PlannerType

agent = PlannerGraph(
    llm = llm,
    db_manager = db_manager,
    book_retriever = DocumentRetriever('data/'),
    planner_type = PlannerType.DIET,
    num_results = 5,
    use_rag_data = False,
    summarize_logs = False # disable log summarization.
).compile()

Making the book retriever ready =====
table is already created=====
Book Retriever is ready ====


In [32]:
plans = []
for i, user in records_df.iterrows():
    response = agent.invoke({"messages": HumanMessage(content=user['query']), 'user_id':i+1}, config={"configurable": {"thread_id": i+1}})
    agent_plan = response['messages'][-1].content
    # time.sleep(50)
    plans.append(agent_plan)
records_df['agent_plan_withoutrag'] = plans


Function 'node_log_summary'
Function 'node_summarize'
Returned: {'messages': [], 'user_id': 1} 
Returned: {'log_summary': None} 
Function 'node_plan'
Returned: {'messages': [AIMessage(content="# Meal Plan Overview\n- Calories (average per day): 1600 kcal  \n- Protein (average per day): 110 g  \n- Carbs (average per day): 130 g  \n- Fat (average per day): 45 g  \n- Fiber (average per day): 25 g  \n- Notes:  \n  This meal plan is designed to balance lean protein intake with controlled carbohydrate portions and plenty of vegetables to support overall health and light activity. Each day includes 3 main meals and 1–2 snacks, emphasizing macro balance with high protein for muscle maintenance, moderate carbs primarily from whole-food sources, and healthy fats to support satiety and energy. The plan suits a 165 cm, 60 kg lightly active female aiming for balanced nutrition with macro focus.\n\n# Daily Meal Tables\n\n## Day 1 (Monday)\n| Meal          | Ingredients                               

In [33]:
records_df.to_csv('./data/evaluation/diet/diet_records.csv', index=False)


### Compare baseline vs agent plan

In [22]:
judges = []
for i, user in records_df.iterrows():
    j = judge(user['query'], master_df['improved_text'][i], user['agent_plan']).model_dump()
    # time.sleep(10)
    judges.append(j)
judges_df = pd.DataFrame(judges)

In [23]:
judges_df

,relevance,completeness,creativity,safety_and_evidence,clarity,overall,rationale
0,Tie,A,A,Tie,B,A,Both plans align well with the user's low-calo...
1,B,A,A,B,B,B,"Plan B aligns better with the user's low-carb,..."
2,Tie,A,A,B,B,B,Both plans align well with the user's goal of ...
3,Tie,A,A,Tie,B,A,Both plans align well with the user's goals of...
4,A,A,A,Tie,B,A,Plan A aligns more closely with the user's goa...
5,A,Tie,B,B,B,B,Plan A offers a higher protein percentage (157...
6,A,A,A,Tie,B,A,Plan A aligns more closely with the user's low...
7,A,B,B,B,B,B,Plan A aligns well with the user's high-protei...
8,A,A,A,B,B,A,Plan A aligns more closely with the user's hig...
9,A,A,A,Tie,Tie,A,Plan A aligns more closely with the user's sta...


In [ ]:
judges_df.to_csv('./data/evaluation/diet/diet_judges_1.csv', index=False)

### Compare RAG-free Agent vs RAG Agent

In [34]:
judges = []
for i, user in records_df.iterrows():
    j = judge(user['query'], user['agent_plan_withoutrag'], user['agent_plan']).model_dump()
    # time.sleep(10)
    judges.append(j)
judges_df = pd.DataFrame(judges)

In [36]:
judges_df

,relevance,completeness,creativity,safety_and_evidence,clarity,overall,rationale
0,B,A,A,Tie,Tie,B,Plan B better aligns with the user's stated go...
1,B,A,A,Tie,Tie,B,Plan B better aligns with the user's stated go...
2,B,A,A,Tie,Tie,B,Plan B better aligns with the user's stated go...
3,B,Tie,Tie,B,Tie,B,Plan B better aligns with the user's goal of a...
4,B,Tie,Tie,B,Tie,B,Plan B better aligns with the user's goal of a...
5,B,B,B,B,Tie,B,Plan B better aligns with the user's high-prot...
6,Tie,Tie,Tie,Tie,Tie,Tie,Both plans align well with the user's low-carb...
7,B,B,B,B,Tie,B,Plan B better aligns with the user's goal of a...
8,B,Tie,B,Tie,A,B,Plan B aligns more closely with the user's goa...
9,B,B,B,Tie,Tie,B,Plan B better aligns with the user's goals by ...


In [37]:
judges_df.to_csv('./data/evaluation/diet/diet_judges_2.csv', index=False)

# Analysis

### baseline vs agent plan

In [ ]:
from sklearn.metrics import cohen_kappa_score

judges_df = pd.read_csv('./data/evaluation/diet/diet_judges_1.csv')

overall_counts = judges_df.query("overall in ['A','B']")['overall'].value_counts()
n_overall = overall_counts.sum()
win_rate_A = overall_counts.get('A', 0) / n_overall
win_rate_B = overall_counts.get('B', 0) / n_overall

print(f"Overall win-rate: Baseline = {win_rate_A:.2%}, Agent = {win_rate_B:.2%} (n={n_overall})")

criteria = ['relevance', 'completeness', 'creativity', 'safety_and_evidence', 'clarity']
breakdown = {}
for crit in criteria:
    mask = judges_df['overall'].isin(['A','B']) & judges_df[crit].isin(['A','B'])
    y_overall = judges_df.loc[mask, 'overall']
    y_crit    = judges_df.loc[mask, crit]

    a = (y_crit == 'A').sum()
    b = (y_crit == 'B').sum()
    ties = len(judges_df) - len(y_crit)

    print(f"\n--- {crit.capitalize()} Breakdown ---")
    print(f"{crit.capitalize()}: Baseline = {a}, Agent = {b}, tie/skipped = {ties}")
    
    if a and b:
        kappa = cohen_kappa_score(y_overall, y_crit, labels=['A', 'B'])
    else:
        kappa = pd.NA
    print(f"Cohen's Kappa (overall vs {crit}): {kappa:.2f}")



Overall win-rate: Baseline = 30.00%, Agent = 70.00% (n=40)

--- Relevance Breakdown ---
Relevance: Baseline = 15, Agent = 11, tie/skipped = 14
Cohen's Kappa (overall vs relevance): 0.63

--- Completeness Breakdown ---
Completeness: Baseline = 8, Agent = 17, tie/skipped = 15
Cohen's Kappa (overall vs completeness): 0.69

--- Creativity Breakdown ---
Creativity: Baseline = 14, Agent = 26, tie/skipped = 0
Cohen's Kappa (overall vs creativity): 0.43

--- Safety_and_evidence Breakdown ---
Safety_and_evidence: Baseline = 0, Agent = 30, tie/skipped = 10
Cohen's Kappa (overall vs safety_and_evidence): <NA>

--- Clarity Breakdown ---
Clarity: Baseline = 5, Agent = 31, tie/skipped = 4
Cohen's Kappa (overall vs clarity): 0.23


### RAG-free Agent vs RAG Agent

In [38]:
from sklearn.metrics import cohen_kappa_score

judges_df = pd.read_csv('./data/evaluation/diet/diet_judges_2.csv')

overall_counts = judges_df.query("overall in ['A','B']")['overall'].value_counts()
n_overall = overall_counts.sum()
win_rate_A = overall_counts.get('A', 0) / n_overall
win_rate_B = overall_counts.get('B', 0) / n_overall

print(f"Overall win-rate: Baseline = {win_rate_A:.2%}, Agent = {win_rate_B:.2%} (n={n_overall})")

criteria = ['relevance', 'completeness', 'creativity', 'safety_and_evidence', 'clarity']
breakdown = {}
for crit in criteria:
    mask = judges_df['overall'].isin(['A','B']) & judges_df[crit].isin(['A','B'])
    y_overall = judges_df.loc[mask, 'overall']
    y_crit    = judges_df.loc[mask, crit]

    a = (y_crit == 'A').sum()
    b = (y_crit == 'B').sum()
    ties = len(judges_df) - len(y_crit)

    print(f"\n--- {crit.capitalize()} Breakdown ---")
    print(f"{crit.capitalize()}: Baseline = {a}, Agent = {b}, tie/skipped = {ties}")
    
    if a and b:
        kappa = cohen_kappa_score(y_overall, y_crit, labels=['A', 'B'])
    else:
        kappa = pd.NA
    print(f"Cohen's Kappa (overall vs {crit}): {kappa:.2f}")



Overall win-rate: Baseline = 7.89%, Agent = 92.11% (n=38)

--- Relevance Breakdown ---
Relevance: Baseline = 2, Agent = 32, tie/skipped = 6
Cohen's Kappa (overall vs relevance): 1.00

--- Completeness Breakdown ---
Completeness: Baseline = 9, Agent = 12, tie/skipped = 19
Cohen's Kappa (overall vs completeness): -0.09

--- Creativity Breakdown ---
Creativity: Baseline = 16, Agent = 13, tie/skipped = 11
Cohen's Kappa (overall vs creativity): 0.06

--- Safety_and_evidence Breakdown ---
Safety_and_evidence: Baseline = 0, Agent = 15, tie/skipped = 25
Cohen's Kappa (overall vs safety_and_evidence): <NA>

--- Clarity Breakdown ---
Clarity: Baseline = 2, Agent = 0, tie/skipped = 38
Cohen's Kappa (overall vs clarity): <NA>


## A sample query, baseline plan, RAG-free agent plan, RAG agent plan

### query

In [5]:
print(records_df.loc[1, 'query'])

I'm looking for a low-carb, high-protein meal plan around 1200 calories to support my weight management and maintain energy levels.


### baseline plan

In [ ]:
display(Markdown(master_df.loc[1, 'improved_text']))

# Meal Plan Overview
- Calories: 1200 kcal (approximate average)
- Protein: ~100g (approximate average)
- Carbohydrates: ~60g (approximate average)
- Fat: ~40g (approximate average)
- Fiber: ~15g (approximate average)
- Notes: This plan is designed for a weekly macro-balanced meal plan with a focus on high protein, moderate fats, and low carbs.

# Daily Meal Tables

| Day | Meal       | Recipe Name                        | Calories | Protein (g) | Carbs (g) | Fat (g) | Fiber (g) |
|-------|------------|-------------------------------------|------------|--------------|-----------|---------|-----------|
| Day 1 | Breakfast  | Mexican scrambled eggs              | 282        | 14           | 13        | 14      | 12        |
|       | Lunch      | Sauteed garlic & herb tomatoes     | 64         | 1            | 3         | 5       | 1         |
|       |            | Teriyaki burgers (1 burger)        | 399        | 37           | 28        | 15      | 1         |
|       | Dinner     | String cheese (2 sticks)           | 165        | 13           | 11        | 3       | 0         |
|       |            | Avocado tuna salad                 | 327        | 30           | 4         | 18      | 6         |
| Day 2 | Breakfast  | Mexican scrambled eggs              | 282        | 14           | 13        | 14      | 12        |
|       | Lunch      | Sauteed garlic & herb tomatoes     | 64         | 1            | 3         | 5       | 1         |
|       |            | Teriyaki burgers (1 burger)        | 399        | 37           | 28        | 15      | 1         |
|       | Dinner     | Sweet potato medallions             | 155        | 2            | 18        | 7       | 3         |
|       |            | Baked chicken with tomatoes & olives | 300      | 40           | 4         | 12      | 3         |
| Day 3 | Breakfast  | Egg in an eggplant                  | 120        | 7            | 1         | 9       | 2         |
|       | Lunch      | Cucumber slices                     | 60         | 3            | 10        | 0       | 2         |
|       |            | Avocado tuna salad sandwich          | 370        | 28           | 25        | 14      | 7         |
|       | Dinner     | Sweet potato medallions             | 155        | 2            | 18        | 7       | 3         |
|       |            | Baked chicken with tomatoes & olives | 300      | 40           | 4         | 12      | 3         |
| Day 4 | Breakfast  | Egg in an eggplant                  | 120        | 7            | 1         | 9       | 2         |
|       | Lunch      | Cucumber slices                     | 60         | 3            | 10        | 0       | 2         |
|       |            | Avocado tuna salad sandwich          | 370        | 28           | 25        | 14      | 7         |
|       | Dinner     | Lentils                            | 87         | 6            | 13        | 0       | 3         |
|       |            | Teriyaki chicken                     | 380        | 55           | 11        | 13      | 0         |
| Day 5 | Breakfast  | Southwest avocado toast             | 193        | 8            | 18        | 7       | 7         |
|       | Lunch      | Buffalo chicken salad               | 356        | 45           | 6         | 16      | 1         |
|       | Dinner     | Tomato and avocado salad            | 117        | 2            | 3         | 9       | 4         |
|       |            | Turkey meatballs                    | 365        | 42           | 4         | 20      | 1         |
| Day 6 | Breakfast  | Southwest avocado toast             | 193        | 8            | 18        | 7       | 7         |
|       | Lunch      | Buffalo chicken salad               | 356        | 45           | 6         | 16      | 1         |
|       | Dinner     | Sunflower seeds                     | 240        | 11           | 3         | 19      | 3         |
|       |            | Spiced chicken tabbouleh bowl       | 241        | 29           | 21        | 4       | 2         |
| Day 7 | Breakfast  | Southwest avocado toast             | 193        | 8            | 18        | 7       | 7         |
|       | Lunch      | Basic chicken breast                | 317        | 50           | 0         | 13      | 0         |
|       |            | Couscous                            | 100        | 4            | 20        | 1       | 1         |
|       | Dinner     | Sunflower seeds                     | 240        | 11           | 3         | 19      | 3         |
|       |            | Spiced chicken tabbouleh bowl       | 241        | 29           | 21        | 4       | 2         |

# Grocery List

## Spices and Herbs
- Black pepper: 1 g
- Salt: 10 g (1/3 oz)
- Ground cumin: 1 g (1/2 tsp)
- Chili powder: 7 g (2 1/2 tsp)
- Fresh basil: 6 g (12 leaves)
- Garlic powder: 2 g (2 1/4 g)
- Oregano, dried: 1 g (5 dashes)
- Onion powder: 1 g (5 dashes)

## Fats and Oils
- Olive oil: 23 mL (3/4 oz)
- Oil (generic): 55 mL (2 oz)
- Mayonnaise: 34 mL (2 1/4 tbsp)

## Vegetables and Vegetable Products
- Garlic: 2 g (1/2 clove)
- Tomatoes: 444 g (3 1/2 medium)
- Cucumber: 708 g (2 1/3 cucumbers)
- Onion: 98 g (1 medium)
- Sweet potatoes: 210 g (1 sweet potato)
- Eggplant: 120 g (2 slices)
- Fresh spinach: 11 g (3/8 cup)
- Raw celery: 26 g (1 1/2 stalks)
- Fresh parsley: 1 g (1 1/3 sprigs)

## Baked Products
- Hamburger buns: 102 g (2 buns)
- Bread: 224 g (1/2 lb)
- Naan bread: 90 g (1 piece)

## Beef and Poultry
- Ground beef (93% lean): 340 g (3/4 lbs)
- Boneless skinless chicken breast, raw: 1352 g (3 lbs)
- Ground turkey, raw: 181 g (6 1/2 oz)

## Dairy and Egg Products
- String cheese: 56 g (2 sticks)
- Eggs: 7 large (350 g)
- Lowfat flavored Greek yogurt: 300 g (2 containers)

## Fruits and Fruits Juices
- Avocados: 327 g (1 1/2 avocados)
- Lime juice: 16 mL (1/2 fl oz)
- Green olives: 53 g (12 large)
- Lemon juice: 3 mL (1/2 tsp)

## Fish and Shellfish
- Canned tuna: 301 g (1 3/4 cans)

## Soups, Sauces, and Gravies
- Salsa: 54 g (3 tbsp)
- Frank's Red Hot sauce: 68 mL (1/4 cup)
- Teriyaki sauce: 90 mL (4 oz)

## Legumes and Legume Products
- Black beans: 220 g (1/2 can)
- Lentils, raw: 24 g (2 tbsp)
- Refried beans: 182 g (3/4 cup)

## Beverages
- Water: 119 mL (1/2 cup)

## Nut and Seed Products
- Sunflower kernels: 76 g (2 2/3 oz)

## Cereal Grains and Pasta
- Instant couscous, flavored: 82 g (1/2 box)

# Recipes

## Mexican Scrambled Eggs
- Ingredients:
  - Eggs: 2 large
  - Black beans: 1/2 can
  - Salsa: 3 tbsp
  - Avocados: 1/2
  - Ground cumin: 2 dashes
  - Chili powder: 4 dashes
  - Oil: 1/2 tsp
- Instructions:
  1. Cook beans and spices in a small pan over low heat.
  2. Scramble eggs in a bowl, cook in a skillet with oil until set.
  3. Serve eggs with beans, avocado, and salsa.
- Nutrition: 282 calories, 14g protein, 13g carbs, 14g fat, 12g fiber

## Egg in an Eggplant
- Ingredients:
  - Eggs: 2
  - Eggplant: 2 slices
  - Oil: 2 tsp
- Instructions:
  1. Cook eggplant slices in oil until soft.
  2. Cut a hole in each slice, crack an egg into each, cook until done.
  3. Serve with seasoning.
- Nutrition: 120 calories, 7g protein, 9g fat, 1g carbs, 2g fiber

## Southwest Avocado Toast
- Ingredients:
  - Bread: 3 slices
  - Eggs: 3
  - Refried beans: 3/4 cup
  - Guacamole: 6 tbsp
  - Spinach: 11g
  - Oil: 1/4 tsp
- Instructions:
  1. Toast bread.
  2. Spread refried beans, top with guacamole and spinach.
  3. Cook eggs as desired.
- Nutrition: ~193 calories, 8g protein, 7g fat, 18g carbs, 7g fiber

## Sauteed Garlic & Herb Tomatoes
- Ingredients:
  - Cherry tomatoes: 149g
  - Olive oil: 11 mL
  - Garlic: 1/2 clove
  - Italian seasoning: 3 dashes
  - Salt: 2 dashes
- Instructions:
  1. Sauté garlic in oil.
  2. Add tomatoes and seasonings, cook 5-7 mins.
  3. Serve warm.
- Nutrition: 64 calories, 1g protein, 5g fat, 3g carbs, 1g fiber

## Teriyaki Burgers
- Ingredients:
  - Ground beef (93% lean): 340g
  - Hamburger buns: 2
  - Teriyaki sauce: 2 tbsp
  - Cucumber slices: 8 slices
- Instructions:
  1. Form beef into patties, cook in skillet.
  2. Brush with teriyaki sauce.
  3. Assemble with buns and cucumber.
- Nutrition: 399 calories, 37g protein, 15g fat, 28g carbs, 1g fiber

## Avocado Tuna Salad
- Ingredients:
  - Canned tuna: 1 can
  - Avocados: 1/2
  - Lime juice: 1 tsp
  - Minced onion: 1/4 small
  - Salt & pepper
  - Mixed greens
  - Tomatoes: chopped
- Instructions:
  1. Mix tuna, avocado, lime, onion, salt, pepper.
  2. Serve on greens with tomatoes.
- Nutrition: ~370 calories, 28g protein, 14g carbs, 14g fat, 7g fiber

## Sweet Potato Medallions
- Ingredients:
  - Sweet potato: 1
  - Oil: 1 tbsp
- Instructions:
  1. Slice sweet potato into thin medallions.
  2. Coat with oil, bake at 425°F for 30 mins.
  3. Serve warm.
- Nutrition: 155 calories, 2g protein, 7g fat, 18g carbs, 3g fiber

## Baked Chicken with Tomatoes & Olives
- Ingredients:
  - Chicken breast: 6 oz
  - Cherry tomatoes: 12
  - Green olives: 12
  - Olive oil: 2 tsp
  - Basil: 6 leaves
  - Seasonings: salt, pepper, chili powder
- Instructions:
  1. Season chicken, place in dish with tomatoes, olives, basil.
  2. Bake at 425°F for 25 mins.
  3. Serve hot.
- Nutrition: 300 calories, 40g protein, 12g fat, 4g carbs, 3g fiber

## Lentils
- Ingredients:
  - Lentils: 2 tbsp
  - Water: 1/2 cup
  - Salt: 1/4 dash
- Instructions:
  1. Rinse lentils.
  2. Simmer in water with salt for 20-30 mins until tender.
  3. Serve.
- Nutrition: 87 calories, 6g protein, 0g fat, 13g carbs, 3g fiber

## Teriyaki Chicken
- Ingredients:
  - Chicken: 8 oz
  - Teriyaki sauce: 4 tbsp
  - Oil: 1/2 tbsp
- Instructions:
  1. Cook chicken in oil until done.
  2. Add teriyaki sauce, heat through.
  3. Serve.
- Nutrition: 380 calories, 55g protein, 13g fat, 11g carbs

## Tomato and Avocado Salad
- Ingredients:
  - Tomato: 1/4 medium
  - Avocado: 1/4
  - Onion: 1/2 tbsp
  - Lime juice: 1/2 tbsp
  - Olive oil: 2 mL
  - Seasonings: salt, pepper, garlic powder
- Instructions:
  1. Mix onion and lime juice.
  2. Add avocado, tomato, oil, seasonings.
  3. Chill and serve.
- Nutrition: 117 calories, 2g protein, 9g fat, 3g carbs, 4g fiber

## Turkey Meatballs
- Ingredients:
  - Ground turkey: 6.5 oz
  - Onion powder, garlic powder, oregano, salt: small amounts
  - Nonfat Greek yogurt: 3 tbsp
  - Oil: 1 tsp
- Instructions:
  1. Mix ingredients, form into 8 meatballs.
  2. Cook in skillet until browned and cooked through.
  3. Serve.
- Nutrition: 365 calories, 42g protein, 20g fat, 4g carbs, 1g fiber

## Spiced Chicken Tabbouleh Bowl
- Ingredients:
  - Chicken breast: 4 oz
  - Couscous: 1/6 box
  - Cucumber, chopped
  - Tomato, chopped
  - Parsley
  - Lemon juice
  - Oil, cumin
- Instructions:
  1. Cook chicken with spices.
  2. Prepare couscous.
  3. Mix ingredients, assemble bowl.
  4. Drizzle lemon juice.
- Nutrition: 241 calories, 29g protein, 4g fat, 21g carbs, 2g fiber

# Notes
- This plan emphasizes high-protein, low-carb meals with balanced fats.
- Recipes are designed for easy preparation and meal prep.
- Adjust seasonings and ingredients as needed for taste and dietary preferences.
- For next week, generate your personalized plan at [Strongr Fastr](https://www.strongrfastr.com).

### RAG agent plan

In [6]:
display(Markdown(records_df.loc[1, 'agent_plan']))

# Meal Plan Overview
- Calories (average per day): 1200 kcal  
- Protein (average per day): 100 g  
- Carbs (average per day): 80 g  
- Fat (average per day): 50 g  
- Fiber (average per day): 25 g  
- Notes:  
  This low-carb, high-protein plan is designed to support weight management with moderate caloric intake and focus on sustaining energy levels through balanced macros. Meals include lean proteins, healthy fats, and fibrous vegetables while minimizing starchy carbs and sugars. Each day has 3 main meals plus 1 optional snack. Portions and ingredient choices aim for accessibility and variety without repetition more than twice per week.

# Daily Meal Tables
## Day 1 (Monday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | 3 scrambled eggs with spinach (1 cup) cooked in 1 tsp olive oil | 280      | 25          | 4         | 19      |
| Snack         | Greek yogurt (150g, plain, nonfat) with 10 almonds             | 160      | 15          | 6         | 8       |
| Lunch         | Grilled chicken breast (150g), mixed greens salad (2 cups), 1 tbsp olive oil, lemon dressing | 320      | 40          | 8         | 12      |
| Dinner        | Baked salmon (120g), steamed broccoli (1.5 cups), 1 tsp butter  | 350      | 38          | 5         | 14      |
| Optional Snack| Cucumber slices with 2 tbsp hummus                              | 90       | 2           | 4         | 3       |

## Day 2 (Tuesday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Protein smoothie: whey protein (25g), unsweetened almond milk (1 cup), half avocado, spinach (1 cup) | 300      | 35          | 8         | 15      |
| Snack         | Boiled egg (1 large) and 1 small apple                         | 140      | 7           | 15        | 5       |
| Lunch         | Turkey breast slices (150g), avocado (half), mixed salad greens, cherry tomatoes | 320      | 38          | 7         | 14      |
| Dinner        | Stir-fried tofu (150g) with mixed low-carb veggies (bell peppers, zucchini, mushrooms), 1 tsp sesame oil | 320      | 30          | 8         | 15      |
| Optional Snack| Celery sticks with 2 tbsp natural peanut butter                | 120      | 5           | 6         | 7       |

## Day 3 (Wednesday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Omelette: 3 eggs, mushrooms, and shredded low-fat cheese       | 290      | 28          | 5         | 18      |
| Snack         | Cottage cheese (150g) with sliced cucumber                      | 150      | 20          | 5         | 3       |
| Lunch         | Grilled shrimp (150g), zucchini noodles (1.5 cups), pesto sauce (1 tbsp) | 330      | 40          | 7         | 13      |
| Dinner        | Lean beef stir-fry (150g) with broccoli and snap peas          | 340      | 38          | 8         | 14      |
| Optional Snack| 1 small handful walnuts                                          | 90       | 3           | 2         | 9       |

## Day 4 (Thursday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Chia pudding (3 tbsp chia seeds) made with unsweetened almond milk, topped with 5 raspberries | 280      | 15          | 8         | 18      |
| Snack         | Hard boiled egg (1 large)                                       | 70       | 6           | 1         | 5       |
| Lunch         | Grilled chicken salad: 150g chicken, kale (2 cups), avocado (quarter), olive oil & vinegar dressing | 320      | 40          | 7         | 13      |
| Dinner        | Oven-baked cod (150g), asparagus (1.5 cups), olive oil drizzle | 350      | 38          | 6         | 14      |
| Optional Snack| 1 small bell pepper sliced with 2 tbsp guacamole                | 90       | 2           | 6         | 6       |

## Day 5 (Friday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Greek yogurt (150g) with flax seeds (1 tbsp) and 5 strawberries | 280      | 20          | 10        | 12      |
| Snack         | Turkey jerky (30g)                                              | 90       | 12          | 3         | 1       |
| Lunch         | Tuna salad: tuna in water (120g), mixed greens, olives (5), olive oil dressing | 320      | 38          | 4         | 13      |
| Dinner        | Roasted turkey breast (150g), cauliflower mash (1 cup), green beans (1 cup) | 350      | 40          | 7         | 15      |
| Optional Snack| 10 raw almonds                                                  | 70       | 3           | 2         | 6       |

## Day 6 (Saturday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Smoothie: whey protein (25g), unsweetened coconut milk (1 cup), 1 tbsp peanut butter, handful spinach | 320      | 35          | 6         | 16      |
| Snack         | 1 boiled egg                                                    | 70       | 6           | 1         | 5       |
| Lunch         | Grilled salmon (150g), arugula and avocado salad with olive oil dressing | 350      | 40          | 6         | 15      |
| Dinner        | Chicken stir-fried (150g) with bok choy, mushrooms, bell peppers, 1 tsp olive oil | 320      | 38          | 7         | 14      |
| Optional Snack| 1 cup sliced cucumber with 2 tbsp tzatziki                     | 70       | 3           | 3         | 3       |

## Day 7 (Sunday)
| Meal          | Ingredients                                                     | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|-----------------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Veggie omelette: 3 eggs, onions, tomatoes, 1 oz feta cheese    | 300      | 28          | 6         | 17      |
| Snack         | Greek yogurt (100g) with 1 tbsp chia seeds                      | 140      | 12          | 6         | 6       |
| Lunch         | Roast beef slices (150g), side salad with mixed greens, olive oil | 320    | 40          | 5         | 13      |
| Dinner        | Baked chicken thighs (150g) with roasted Brussels sprouts (1.5 cups), 1 tsp olive oil | 320      | 40          | 7         | 13      |
| Optional Snack| Small handful pumpkin seeds                                     | 80       | 3           | 3         | 7       |

# Grocery List
- Proteins:  
  - Eggs  
  - Chicken breast  
  - Salmon  
  - Cod  
  - Turkey breast  
  - Tuna (canned in water)  
  - Shrimp  
  - Lean beef  
  - Tofu  
  - Greek yogurt (plain, nonfat)  
  - Cottage cheese  
  - Whey protein powder  
  - Turkey jerky  
  - Roast beef slices  
- Carbs & Grains:  
  - Zucchini (for noodles)  
  - Cauliflower (for mash)  
  - Mixed salad greens (kale, spinach, arugula, lettuce)  
  - Bell peppers  
  - Broccoli  
  - Mushrooms  
  - Asparagus  
  - Green beans  
  - Brussels sprouts  
  - Cucumber  
  - Tomatoes  
  - Onions  
  - Apples (small)  
  - Strawberries  
  - Raspberries  
- Vegetables & Fruits:  
  - Avocado  
  - Lemons  
- Others:  
  - Olive oil  
  - Butter  
  - Sesame oil  
  - Natural peanut butter  
  - Hummus  
  - Tzatziki  
  - Walnuts  
  - Almonds  
  - Pumpkin seeds  
  - Flax seeds  
  - Chia seeds  
  - Feta cheese  
  - Pesto sauce (or ingredients to make: basil, pine nuts, olive oil, garlic)

# Recipes
## Recipe 1: Scrambled Eggs with Spinach
- Ingredients:  
  - 3 large eggs  
  - 1 cup fresh spinach  
  - 1 tsp olive oil  
  - Salt and pepper to taste  
- Instructions:  
  1. Heat olive oil in a nonstick pan over medium heat.  
  2. Add spinach and sauté until wilted (about 2 minutes).  
  3. Beat eggs with salt and pepper, pour into pan.  
  4. Stir gently until eggs are fully cooked but still moist. Serve warm.

## Recipe 2: Grilled Chicken Salad
- Ingredients:  
  - 150g chicken breast  
  - 2 cups mixed salad greens (kale, spinach, arugula)  
  - 1 tbsp olive oil  
  - Juice of half a lemon  
  - Salt and pepper to taste  
- Instructions:  
  1. Season chicken breast with salt and pepper and grill until cooked through (~6-7 mins each side).  
  2. Toss mixed greens with olive oil and lemon juice.  
  3. Slice chicken and place over salad. Serve immediately.

## Recipe 3: Baked Salmon with Steamed Broccoli
- Ingredients:  
  - 120g salmon fillet  
  - 1.5 cups broccoli florets  
  - 1 tsp butter  
  - Lemon wedge  
  - Salt and pepper to taste  
- Instructions:  
  1. Preheat oven to 375°F (190°C).  
  2. Place salmon on baking tray, season with salt, pepper, and lemon wedge. Bake 15-20 minutes until cooked.  
  3. Steam broccoli until tender (~5 minutes).  
  4. Toss broccoli with butter and serve along with salmon.

## Recipe 4: Tofu Stir-fry
- Ingredients:  
  - 150g firm tofu, cubed  
  - 1 cup mixed vegetables (bell peppers, zucchini, mushrooms)  
  - 1 tsp sesame oil  
  - 1 tbsp low-sodium soy sauce (optional)  
- Instructions:  
  1. Heat sesame oil in pan over medium heat.  
  2. Add tofu cubes and cook until golden on all sides (5-7 minutes).  
  3. Add vegetables and stir-fry until tender-crisp.  
  4. Season with soy sauce if desired; serve hot.

---

This plan offers sufficient protein and moderate carbs with healthy fats to support satiety and energy throughout your day while keeping calories around 1200. Let me know if you want recipes or substitutions for any meals!

### RAG-free agent plan

In [35]:
display(Markdown(records_df.loc[1, 'agent_plan_withoutrag']))

# Meal Plan Overview
- Calories (average per day): 2000 kcal  
- Protein (average per day): 100 g  
- Carbs (average per day): 220 g  
- Fat (average per day): 65 g  
- Fiber (average per day): 30 g  
- Notes:  
  This plan offers a balanced macronutrient profile tailored to moderate activity level and maintenance weight goals. Meals emphasize whole foods with a good mix of protein, complex carbs, healthy fats, and fiber. Variety ensures nutritional coverage and enjoyment.

# Daily Meal Tables

## Day 1 (Monday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Greek yogurt (150g), mixed berries (100g), granola (40g) | 350      | 20          | 40        | 7       |
| Snack         | Apple (1 medium), almonds (15g)                      | 180      | 4           | 22        | 9       |
| Lunch         | Grilled chicken breast (120g), quinoa (1 cup cooked), steamed broccoli (100g), olive oil (1 tbsp) | 550      | 45          | 45        | 15      |
| Dinner        | Baked salmon (150g), sweet potato (150g), sautéed spinach (100g) with garlic, olive oil (1 tbsp) | 600      | 40          | 40        | 20      |
| Optional Snack| Cottage cheese (100g) with sliced cucumber            | 120      | 15          | 5         | 3       |

## Day 2 (Tuesday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Oatmeal (50g) with skim milk (200ml), banana (1 medium), peanut butter (1 tbsp) | 400      | 15          | 55        | 10      |
| Snack         | Carrot sticks (100g) with hummus (50g)               | 180      | 5           | 15        | 10      |
| Lunch         | Turkey wrap: whole wheat tortilla, turkey breast (120g), mixed greens, tomato, avocado (30g) | 520      | 40          | 45        | 15      |
| Dinner        | Stir-fried tofu (150g) with mixed vegetables (carrot, bell pepper, snap peas - 150g), brown rice (1 cup cooked) | 620      | 35          | 55        | 18      |
| Optional Snack| Mixed nuts (20g)                                      | 180      | 5           | 5         | 16      |

## Day 3 (Wednesday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Smoothie: protein powder (1 scoop), spinach (30g), mixed frozen berries (100g), almond milk (200ml), flaxseeds (1 tbsp) | 380      | 30          | 35        | 8       |
| Snack         | Hard boiled eggs (2 eggs)                            | 140      | 12          | 1         | 10      |
| Lunch         | Lentil soup (1.5 cups), whole-grain bread (1 slice), side salad (lettuce, tomato, cucumbers, light vinaigrette) | 520      | 30          | 50        | 10      |
| Dinner        | Grilled lean beef steak (150g), roasted Brussels sprouts and carrots (150g), small baked potato (100g) | 620      | 45          | 40        | 20      |
| Optional Snack| Greek yogurt (100g) with a drizzle of honey          | 150      | 10          | 15        | 0       |

## Day 4 (Thursday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Whole grain toast (2 slices), scrambled eggs (2 eggs), avocado (50g) | 450      | 25          | 35        | 20      |
| Snack         | Orange (1 medium), sunflower seeds (15g)             | 170      | 5           | 25        | 8       |
| Lunch         | Chickpea salad: chickpeas (1 cup cooked), cucumber, tomato, red onion, feta cheese (30g), olive oil (1 tbsp), lemon juice | 520      | 25          | 45        | 18      |
| Dinner        | Baked cod (150g), wild rice (1 cup cooked), steamed asparagus (100g), olive oil (1 tbsp) | 600      | 40          | 40        | 15      |
| Optional Snack| Dark chocolate square (20g), walnuts (15g)            | 150      | 3           | 10        | 12      |

## Day 5 (Friday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Chia pudding (chia seeds 3 tbsp, coconut milk 150ml), mango chunks (100g) | 400      | 12          | 40        | 18      |
| Snack         | Celery sticks (100g) with peanut butter (1 tbsp)    | 180      | 5           | 10        | 14      |
| Lunch         | Grilled shrimp (150g) with whole wheat pasta (1 cup cooked), tomato basil sauce, side mixed greens | 550      | 45          | 50        | 15      |
| Dinner        | Turkey chili (turkey mince 150g, beans, tomatoes, peppers), small corn tortilla (1) | 600      | 40          | 50        | 15      |
| Optional Snack| Cottage cheese (100g) with pineapple chunks (50g)     | 120      | 15          | 8         | 2       |

## Day 6 (Saturday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Protein pancakes (oats, eggs, protein powder), topped with fresh strawberries (100g), maple syrup (1 tbsp) | 420      | 30          | 50        | 7       |
| Snack         | Mixed berry bowl (150g), handful of pumpkin seeds (15g) | 200      | 5           | 25        | 10      |
| Lunch         | Grilled chicken Caesar salad: chicken breast (120g), romaine, Parmesan, light dressing, whole wheat croutons | 530      | 45          | 30        | 18      |
| Dinner        | Vegetable stir-fry with tempeh (150g), soba noodles (1 cup cooked), sesame oil (1 tbsp) | 620      | 40          | 50        | 22      |
| Optional Snack| Greek yogurt (100g) with blueberries (50g)            | 130      | 10          | 12        | 2       |

## Day 7 (Sunday)
| Meal          | Ingredients                                           | Calories | Protein (g) | Carbs (g) | Fat (g) |
|---------------|------------------------------------------------------|----------|-------------|-----------|---------|
| Breakfast     | Avocado toast (whole grain bread 2 slices, 100g avocado), poached eggs (2 eggs) | 480      | 25          | 40        | 22      |
| Snack         | Pear (1 medium), small handful of almonds (15g)     | 180      | 4           | 28        | 9       |
| Lunch         | Baked chicken thigh (150g), couscous (1 cup cooked), roasted mixed vegetables (150g) | 570      | 45          | 45        | 15      |
| Dinner        | Minestrone soup with beans, vegetables, whole grain roll (1 small) | 550      | 30          | 50        | 13      |
| Optional Snack| Dark chocolate square (20g), raspberries (50g)        | 120      | 2           | 15        | 8       |

# Grocery List
- Proteins:
  - Chicken breast (approx. 1.5 kg)
  - Salmon fillet (150g)
  - Lean beef steak (150g)
  - Turkey breast & mince (~270g)
  - Eggs (12)
  - Tofu (150g)
  - Tempeh (150g)
  - Greek yogurt (900g)
  - Cottage cheese (300g)
  - Protein powder (plant or whey)
  - Shrimp (150g)
  - Feta cheese (30g)
  - Parmesan cheese
- Carbs & Grains:
  - Quinoa
  - Brown rice
  - Wild rice
  - Whole wheat bread
  - Whole wheat tortillas
  - Whole wheat pasta
  - Oats
  - Soba noodles
  - Couscous
  - Granola
- Vegetables & Fruits:
  - Mixed berries (fresh or frozen)
  - Bananas
  - Apples
  - Oranges
  - Mango
  - Strawberries
  - Pear
  - Raspberries
  - Sweet potatoes
  - Potatoes
  - Carrots
  - Broccoli
  - Spinach
  - Asparagus
  - Tomatoes
  - Cucumbers
  - Bell peppers
  - Romaine lettuce
  - Mixed salad greens
  - Onions (red)
  - Garlic
  - Brussels sprouts
  - Celery
  - Pumpkin seeds
  - Avocados
- Others:
  - Almonds
  - Walnuts
  - Peanut butter
  - Sunflower seeds
  - Olive oil
  - Sesame oil
  - Hummus
  - Dark chocolate (70%+ cacao)
  - Maple syrup
  - Lemon juice
  - Light vinaigrette dressing

# Recipes

## Recipe 1: Lentil Soup
- Ingredients:  
  - 1 cup dried lentils (or 2 cups cooked)  
  - 1 onion, diced  
  - 2 carrots, diced  
  - 2 celery stalks, diced  
  - 2 cloves garlic, minced  
  - 4 cups vegetable broth  
  - 1 can diced tomatoes  
  - 1 tsp cumin  
  - Salt and pepper to taste  
- Instructions:  
  1. In a large pot, sauté onion, carrots, celery, and garlic until softened.  
  2. Add lentils, broth, tomatoes, and cumin. Bring to boil, then simmer for 25-30 minutes until lentils are tender.  
  3. Season with salt and pepper. Serve hot with whole grain bread.

## Recipe 2: Protein Pancakes
- Ingredients:  
  - 50g oats  
  - 1 scoop protein powder  
  - 2 eggs  
  - ½ banana mashed  
  - ½ tsp baking powder  
- Instructions:  
  1. Blend oats into flour, mix with protein powder and baking powder.  
  2. Add eggs and mashed banana to form batter.  
  3. Cook pancakes on medium heat 2-3 min each side.  
  4. Serve topped with fresh berries and a drizzle of maple syrup.

## Recipe 3: Turkey Chili
- Ingredients:  
  - 150g lean turkey mince  
  - 1 can kidney beans  
  - 1 can diced tomatoes  
  - 1 bell pepper, diced  
  - 1 small onion, diced  
  - 1 clove garlic, minced  
  - 1 tsp chili powder  
  - Salt and pepper to taste  
- Instructions:  
  1. Brown turkey mince with onion and garlic.  
  2. Add bell pepper, beans, tomatoes, and chili powder.  
  3. Simmer 20-25 minutes until flavors meld.  
  4. Serve with a side of corn tortilla.

If you want more recipes or adjustable portions let me know!